In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/')
os.listdir()

For training data

In [ ]:
# VCTK Corpus Path
__CORPUSPATH__ = "/content/drive/MyDrive/new_cycle/StarGANv2-VC/speakers"
# output path
__OUTPATH__ = "/content/drive/MyDrive/new_cycle/StarGANv2-VC/Data"

For target voice

In [ ]:
# VCTK Corpus Path
__CORPUSPATH__ = "/content/drive/MyDrive/new_cycle/StarGANv2-VC/test_voice_data/trump_target_voice/"
# output path
__OUTPATH__ = "/content/drive/MyDrive/new_cycle/StarGANv2-VC/test_voice_data/trump_voice_preprocessed/"

In [ ]:
!pip install pydub

In [ ]:
import os
from scipy.io import wavfile
from pydub import AudioSegment

from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def split(sound):
    dBFS = sound.dBFS
    chunks = split_on_silence(sound,
        min_silence_len = 100,
        # if you need the full voice in one audio
        # min_silence_len = 10000,
        silence_thresh = dBFS-16,
        keep_silence = 100
    )
    return chunks

def combine(_src):
    audio = AudioSegment.empty()
    for i,filename in enumerate(os.listdir(_src)):
        if filename.endswith('.wav'):
            filename = os.path.join(_src, filename)
            audio += AudioSegment.from_wav(filename)
    return audio

def save_chunks(chunks, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 0

    target_length = 5 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length,
            # we can start a new one
            output_chunks.append(chunk)

    for chunk in output_chunks:
        chunk = chunk.set_frame_rate(24000)
        chunk = chunk.set_channels(1)
        counter = counter + 1
        chunk.export(os.path.join(directory, str(counter) + '.wav'), format="wav")



In [ ]:
speakers = [0 , 1]

In [ ]:
for p in speakers:
    directory = __OUTPATH__ + '/p' + str(p)
    if not os.path.exists(directory):
        audio = combine(__CORPUSPATH__ + '/'+ str(p))
        chunks = split(audio)
        save_chunks(chunks, directory)

In [ ]:
data_list = []
for path, subdirs, files in os.walk(__OUTPATH__):
    for name in files:
        if name.endswith(".wav"):
            speaker = int(path.split('/')[-1].replace('p', ''))
            if speaker in speakers:
                data_list.append({"Path": os.path.join(path, name), "Speaker": int(speakers.index(speaker)) + 1})

import pandas as pd

data_list = pd.DataFrame(data_list)
data_list = data_list.sample(frac=1)

import random

split_idx = round(len(data_list) * 0.1)

test_data = data_list[:split_idx]
train_data = data_list[split_idx:]

In [ ]:
test_data

,Path,Speaker
30,./Data/p0/31.wav,1
110,./Data/p1/40.wav,2
113,./Data/p1/43.wav,2
126,./Data/p1/56.wav,2
82,./Data/p1/12.wav,2
50,./Data/p0/51.wav,1
54,./Data/p0/55.wav,1
146,./Data/p1/76.wav,2
97,./Data/p1/27.wav,2
124,./Data/p1/54.wav,2


In [ ]:
file_str = ""
for index, k in train_data.iterrows():
    file_str += k['Path'] + "|" +str(k['Speaker'] - 1)+ '\n'
text_file = open(__OUTPATH__ + "/train_list.txt", "w")
text_file.write(file_str)
text_file.close()

file_str = ""
for index, k in test_data.iterrows():
    file_str += k['Path'] + "|" + str(k['Speaker'] - 1) + '\n'
text_file = open(__OUTPATH__ + "/val_list.txt", "w")
text_file.write(file_str)
text_file.close()

In [ ]:
# ext for preprocess the target data only
directory = __OUTPATH__

audio = combine(__CORPUSPATH__)
chunks = split(audio)
save_chunks(chunks, directory)

In [ ]:
from pydub import AudioSegment
wav_file = AudioSegment.from_file(file = "/content/drive/MyDrive/new_cycle/StarGANv2-VC/test_voice_data/trump_voice_preprocessed/1.wav",format = "wav")

In [ ]:
wav_file